In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from PIL import Image

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [13]:
train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):           # x.shape = N, 3, 32, 32
        x = F.relu(self.conv1(x))   # x.shape = N, 16, 30, 30
        x = self.pool(x)            # x.shape = N, 16, 15, 15
        x = F.relu(self.conv2(x))   # x.shape = N, 32, 13, 13
        x = self.pool(x)            # x.shape = N, 32, 6, 6
        x = F.relu(self.conv3(x))   # x.shape = N, 64, 4, 4
        x = self.pool(x)            # x.shape = N, 64, 2, 2
        x = torch.flatten(x, 1)     # x.shape = N, 64 * 2 * 2
        x = F.relu(self.fc1(x))     # x.shape = N, 128
        x = F.relu(self.fc2(x))     # x.shape = N, 64
        x = self.fc3(x)             # x.shape = N, 10
        return x

model = Net().to(device)